In [1]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
import pandas as pd
from langchain.chains import LLMChain
from langfuse.callback import CallbackHandler
import os
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser

load_dotenv()

handler = CallbackHandler(os.environ.get("LANGFUSE_PUBLIC_KEY"), os.environ.get("LANGFUSE_SECRET_KEY"))

In [2]:
MODEL = "gpt-3.5-turbo"
OUTPUT_FOLDER = "../generated_GUIs/"
IMAGE_INPUT_FOLDER = "../data/"

s2w = pd.read_csv("../data/Screen2Words.csv")
s2w = s2w.drop('Unnamed: 0', axis=1)
sample = pd.read_csv("../data/s2w_sample.csv")
sample = sample.drop("Unnamed: 0", axis=1)

In [3]:
from notebooks.prompts import SUMMARY_PROMPT, CREATION_PROMPT, COT_CREATION_PROMPT, TOT_CREATION_PROMPT


#Sequential chain, of summarizing the Screen2Words Summaries into one and creating a GUI
def s2w_summarized(id, prompt_type="",temp=0.1, summary_prompt=SUMMARY_PROMPT, creation_prompt=CREATION_PROMPT):

    summaries = s2w[s2w["screenId"]==id]["summary"].values

    llm = ChatOpenAI(model_name=MODEL, temperature=temp)
    
    summary_chain = summary_prompt | llm | StrOutputParser()
    creation_chain = creation_prompt | llm | StrOutputParser()
    
    chain = {"summary": summary_chain} | RunnablePassthrough.assign(creation=creation_chain)
    
    summaries_str = "\n".join([f"Summary {i+1}: {summary}" for i, summary in enumerate(summaries)])
    
    try: 
        response = chain.invoke({"summaries": summaries_str}, config={"callbacks":[handler]})
    except Exception as e:
        print(e)
    
    print(response)
    if prompt_type == "COT":
        output_file = OUTPUT_FOLDER + f"/summarized/Cot/{id}.html"
    elif prompt_type == "TOT":
        output_file = OUTPUT_FOLDER + f"/summarized/Tot/{id}.html"
    else:
        output_file = OUTPUT_FOLDER + f"/summarized/vanilla/{id}.html"

    with open(output_file, 'w') as f:
        f.write(response.get('creation'))
    return response

In [5]:
create_gui(486, prompt=S2W_CREATION_PROMPT)

<!DOCTYPE html>
<html>
<head>
    <title>Mobile Screen</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f2f2f2;
        }
        
        .container {
            width: 100%;
            max-width: 400px;
            margin: 0 auto;
            padding: 20px;
            background-color: #fff;
            border-radius: 10px;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
        }
        
        h1 {
            text-align: center;
            margin-top: 0;
        }
        
        p {
            margin-bottom: 10px;
        }
        
        .alert {
            background-color: #f8d7da;
            color: #721c24;
            padding: 10px;
            border-radius: 5px;
            margin-bottom: 20px;
        }
        
        .popup {
            position: fixed;
            top: 50%;
            left: 50%;
            transform: translate(-50%, -50%);
            width: 300px;
            padding: 2

In [23]:
#pipeline
def creation_pipeline():
    screens = sample["screenId"].head(10).values
    for screen_id in screens:
        s2w_summarized(screen_id)
        print(f"{screen_id}: creation done")

In [24]:
creation_pipeline()

{'summary': 'The screen is a pop-up or page that shows various sharing options for social apps. It allows the user to share an episode through different social apps.', 'creation': '<!DOCTYPE html>\n<html>\n<head>\n    <title>Share Episode</title>\n    <style>\n        body {\n            font-family: Arial, sans-serif;\n            background-color: #f2f2f2;\n        }\n        \n        .container {\n            width: 300px;\n            margin: 0 auto;\n            padding: 20px;\n            background-color: #fff;\n            border-radius: 5px;\n            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.3);\n        }\n        \n        h1 {\n            text-align: center;\n            color: #333;\n        }\n        \n        .social-icons {\n            display: flex;\n            justify-content: space-around;\n            margin-top: 20px;\n        }\n        \n        .social-icons a {\n            text-decoration: none;\n            color: #333;\n        }\n        \n        .soc

In [14]:
s2w_summarized(1264)

{'summary': 'The screen is a fitness app page that offers various workout options to set. It also includes a timer and different features for tracking exercises. Additionally, there is a pop-up that provides information about body workouts. Users can start exercises and workouts from this page.', 'creation': '<!DOCTYPE html>\n<html>\n<head>\n    <title>Fitness App</title>\n    <style>\n        body {\n            font-family: Arial, sans-serif;\n            background-color: #f2f2f2;\n        }\n        \n        .container {\n            max-width: 600px;\n            margin: 0 auto;\n            padding: 20px;\n            background-color: #fff;\n            border-radius: 10px;\n            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);\n        }\n        \n        h1 {\n            text-align: center;\n            color: #333;\n        }\n        \n        .workout-options {\n            display: flex;\n            flex-wrap: wrap;\n            justify-content: space-between;\n        

{'summary': 'The screen is a fitness app page that offers various workout options to set. It also includes a timer and different features for tracking exercises. Additionally, there is a pop-up that provides information about body workouts. Users can start exercises and workouts from this page.',
 'creation': '<!DOCTYPE html>\n<html>\n<head>\n    <title>Fitness App</title>\n    <style>\n        body {\n            font-family: Arial, sans-serif;\n            background-color: #f2f2f2;\n        }\n        \n        .container {\n            max-width: 600px;\n            margin: 0 auto;\n            padding: 20px;\n            background-color: #fff;\n            border-radius: 10px;\n            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);\n        }\n        \n        h1 {\n            text-align: center;\n            color: #333;\n        }\n        \n        .workout-options {\n            display: flex;\n            flex-wrap: wrap;\n            justify-content: space-between;\n       

In [6]:
s2w[s2w["screenId"]==486]["summary"].values


array(['alert message displaying social apps',
       'display of a pop up with share options',
       'page displaying multiple share options',
       'pop-up showing different sharing options',
       'share episode through social app'], dtype=object)